In [1]:
# Yuuno = vapoursyn + jupyter 
# https://yuuno.encode.moe/
# pip install yuuno
# python -m yuuno.console_scripts jupyter install
# from vapoursynth import core # vs core 已经被yuuno import
import havsfunc as haf
import os
%load_ext yuuno

In [2]:
# 测试用视频
video_file = "test.avi"
print(os.path.exists(video_file))

True


In [3]:
# class Core
# vapoursynth 性能参数
# num_thread  默认就是逻辑处理器数 multiprocessing.cpu_count()
# max_cache_size 内存用完后的磁盘上的cache大小(mb)
# print(core.list_functions()) Returns a dict containing all loaded plugins and their functions human-readable
print(core.version())
print(core.max_cache_size)
print(core.num_threads)

VapourSynth Video Processing Library
Copyright (c) 2012-2020 Fredrik Mellbin
Core R52
API R3.6
Options: -

4096
12


In [4]:
# 简陋的获取信息的函数
def video_info(video):
    print(video)
    property_set = {"width", "height", "fps", "num_frames"}
    print(video.format)
    for item in property_set & set(dir(video)):
        print(item, end=":")
        print(getattr(video, item))

# vsdb.top Source/Output 分类下的插件
读取视频 -> vapoursynth.VideoNode

In [5]:
"""
1.avisource 
AVISource(string path[, string pixel_type, string fourcc, bint alpha=False])
pixel_type: YV24, YV16, YV12, YV411, YUY2, Y8, RGB32, RGB24, RGB48, P010, P016, P210, P216, v210
FOURCC四字符码 本例中为LAGS
"""
?core.avisource.AVISource
src = core.avisource.AVISource(video_file)
video_info(src)
#src = core.lsmas.LWLibavSource(video_file,fpsnum=0,fpsden=1,decoder="")

VideoNode
	Format: YUV420P8
	Width: 1920
	Height: 1080
	Num Frames: 3977
	FPS: 60
	Flags: NoCache IsCache

Format Descriptor
	Id: 3000010
	Name: YUV420P8
	Color Family: YUV
	Sample Type: Integer
	Bits Per Sample: 8
	Bytes Per Sample: 1
	Planes: 3
	Subsampling W: 1
	Subsampling H: 1

height:1080
num_frames:3977
fps:60
width:1920


In [6]:
"""
2.lsmas
LibavSMASHSource只用于mp4/mov
其他container用LibavSMASHSource
fps=fpsnum/fpsden 比如 29.97=30000/1001 0/1则尝试可变fps
decoder
"""
?core.lsmas.LWLibavSource
# src=core.lsmas.LibavSMASHSource(video_file,fpsnum=0,fpsden=1,decoder="")
src = core.lsmas.LWLibavSource(video_file, fpsnum=0, fpsden=1, decoder="")
video_info(src)

VideoNode
	Format: YUV420P8
	Width: 1920
	Height: 1080
	Num Frames: 3977
	FPS: 60
	Flags: NoCache IsCache

Format Descriptor
	Id: 3000010
	Name: YUV420P8
	Color Family: YUV
	Sample Type: Integer
	Bits Per Sample: 8
	Bytes Per Sample: 1
	Planes: 3
	Subsampling W: 1
	Subsampling H: 1

height:1080
num_frames:3977
fps:60
width:1920


In [7]:
print(core.ffms2.Version())
?core.ffms2.Source()
src = core.ffms2.Source(video_file, fpsnum=0, fpsden=1)
print(video_info(src))

b'2.23.0.0'
Object `core.ffms2.Source()` not found.
VideoNode
	Format: YUV420P8
	Width: 1920
	Height: 1080
	Num Frames: 3977
	FPS: 60
	Flags: NoCache IsCache

Format Descriptor
	Id: 3000010
	Name: YUV420P8
	Color Family: YUV
	Sample Type: Integer
	Bits Per Sample: 8
	Bytes Per Sample: 1
	Planes: 3
	Subsampling W: 1
	Subsampling H: 1

height:1080
num_frames:3977
fps:60
width:1920
None


# 视频的预览与输出

%%vspreview 预览   
%%vspipe 直接输出  
--reset-core --isolate-variables 视频的导入 处理必须在下面的代码里完成  

In [8]:
%%vspreview
src[:30].set_output()

Preview(clip=<vapoursynth.VideoNode object at 0x000001BD3E74B820>)

In [39]:
%%vspipe --y4m | ffmpeg -i pipe: -c:v hevc_nvenc -preset slow -y test.265
src = core.lsmas.LWLibavSource(video_file, fpsnum=0, fpsden=1, decoder="")
#src = haf.ChangeFPS(src, fpsnum=30000, fpsden=1001) #降低fps
src = core.resize.Bilinear(src,1280,720)
src.set_output()

EncodeWidget(length=3977, start_time=1611478950)

In [40]:
# ffmpeg分离转码音频
os.system("ffmpeg -i test.avi -vn -f flac test.flac")

0

# 用mkvtoolnix-gui 合并视频与音频

In [ ]:
# 也许以后搞点命令行